# Libraries

In [ ]:
!pip install "../input/keras-application/Keras_Applications-1.0.8-py3-none-any.whl"
!pip install "../input/efficientnet111/efficientnet-1.1.1-py3-none-any.whl"
#!pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
#!pip install "../input/hpapytorchzoozip/pytorch_zoo-master"
#!pip install "../input/tfexplainforoffline/tf_explain-0.2.1-py3-none-any.whl"

In [ ]:
import gc
gc.collect()

In [ ]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from PIL import Image
from pathlib import Path
%matplotlib inline

import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm.auto import tqdm

# Modules for Deep Learning & Computer Vision

import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import Callback
from efficientnet.keras import EfficientNetB0
from keras.layers import Dense, Flatten
from keras.models import Model, load_model

import cv2
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
from IPython.display import Image, display
from tensorflow.python.framework import ops


# Modules to encode predictions
#from pycocotools import _mask as coco_mask
#import base64
#import zlib
#import typing as t


print("Done--------")

In [ ]:
# to use GPU, check in top right of Kaggle notebook with 3 points, select 'accelerator' and change cpu by "cuda"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# 1. Some hands-on data vizualisation with train set

## 1.1 Vizualising some images

In [ ]:
data_train = pd.read_csv("../input/hpa-single-cell-image-classification/train.csv")
data_train.head(10)

> **Each image is uniquely identified by its "ID" with corresponding labels (most of the time multi-labels)**

In [ ]:
data_train.shape # contains ID of 21806 images which are superposition of 4 chanels (filterred images) RGBY.

In [ ]:
labels = [
'0. Nucleoplasm',
'1. Nuclear membrane',
'2. Nucleoli',
'3. Nucleoli fibrillar center',
'4. Nuclear speckles',
'5. Nuclear bodies',
'6. Endoplasmic reticulum',
'7. Golgi apparatus',
'8. Intermediate filaments',
'9. Actin filaments',
'10. Microtubules',
'11. Mitotic spindle',
'12. Centrosome',
'13. Plasma membrane',
'14. Mitochondria',
'15. Aggresome',
'16. Cytosol',
'17. Vesicles and punctate cytosolic patterns',
'18. Negative'
] 

In [ ]:
def translate(n):
    return labels[int(n)]

list(map(translate,[8,5,0]))

In [ ]:
data_train["Labels_names"] = data_train["Label"].apply(lambda x : x.split("|")).apply(lambda x : list(map(translate,x)))
data_train.head()

In [ ]:
# printing 3 channels corresponding to one image + target (green title)
# n correspond to the number of the image = index in data_train


def image_channels(n):
    fig, ax = plt.subplots(1, 4, figsize=(30, 20), subplot_kw=dict(xticks=[], yticks=[]))
    
    img_blue = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_blue.png".format(data_train.loc[n,"ID"])))
    img_green = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_green.png".format(data_train.loc[n,"ID"])))
    img_red = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_red.png".format(data_train.loc[n,"ID"])))
    img_yellow = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_yellow.png".format(data_train.loc[n,"ID"])))
    
    ax[0].imshow(img_blue)
    ax[0].set_title('Blue filter = Nucleus', size=20)
    ax[1].imshow(img_green)
    ax[1].set_title("{}".format(data_train.loc[n,"Labels_names"]), size=16 , color = "green")
    ax[2].imshow(img_red)
    ax[2].set_title('Red filter = Microtubules', size=20)
    ax[3].imshow(img_yellow)
    ax[3].set_title('Yellow filter = Endoplasmic Reticulum', size=20)

In [ ]:
image_channels(1)

## 1.2 Comparison of target image with merged RGB channels

In [ ]:
def compare(n):
    fig, ax = plt.subplots(1, 2, figsize=(15, 25),subplot_kw=dict(xticks=[], yticks=[]))
    
    img_blue = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_blue.png".format(data_train.loc[n,"ID"])))
    img_green = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_green.png".format(data_train.loc[n,"ID"])))
    img_red = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_red.png".format(data_train.loc[n,"ID"])))
    img_yellow = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_yellow.png".format(data_train.loc[n,"ID"])))
    
    #ax[1].imshow(img_green)
    #ax[1].set_title("{}".format(data_train.loc[n,"Labels_names"]), size=16 , color = "green")
    
    # print a complete image according to Green channel (protein labels)
    img0 = plt.imread(Path("../input/hpa-single-cell-image-classification/train/{}_green.png".format(data_train.loc[n,"ID"])))
    ax[0].imshow(img0)
    ax[0].set_title("Image G / Target: {}".format(data_train.loc[n,"Labels_names"]), size=16 , color = "green")
    
    # print a complete image according to 3 channels Red Green Blue
    img1 = np.dstack((img_red, img_green, img_blue))
    ax[1].imshow(img1)
    ax[1].set_title("Image RGB / Target: {}".format(data_train.loc[n,"Labels_names"]), size=16 , color = "green")
       
# I think important to recall that blue + yellow = green, thus when we put together channels R/Y/B, protein of interest appears ?

In [ ]:
compare(1)

## 1.3 Define methods to open 4-channel images from train/test set

In [ ]:
TEST_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/test/'
TRAIN_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/train/'
IMG_HEIGHT = IMG_WIDTH = 128 # other good parameter is 512

print("Done--------")

In [ ]:
# function that reads RGBY image (4-channels) with resizing
def open_rgby(image_id): 
    colors = ['red','green','blue','yellow']
    img = [cv2.imread(os.path.join(TRAIN_IMGS_FOLDER , f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE) for color in colors]
    img = np.stack(img, axis=-1)
    img_resized = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    img_resized=img_resized/255
    return img_resized

# same function without resizing for test set
def open_rgby_test(image_id): 
    colors = ['red','green','blue','yellow']
    img = [cv2.imread(os.path.join(TEST_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE) for color in colors]
    img = np.stack(img, axis=-1)
    return img


print("Done--------")

In [ ]:
# With 4 channels RGBY (resized to 128x128)
img = open_rgby("5fb643ee-bb99-11e8-b2b9-ac1f6b6435d0")
plt.imshow(img)


## 1.4 Image Segmentation

Now we will create our first multi-label mask. The task is not easy at all, but fortunately, the Human Cell Group provide us a powerful module "HPACellSegmentator" already tested for a previous challenge. The module is available on GitHub at https://github.com/CellProfiling/HPA-Cell-Segmentation

In [ ]:
!pip install https://github.com/CellProfiling/HPA-Cell-Segmentation/archive/master.zip

In [ ]:
!pip install "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

In [ ]:


#load of the pre-trained weights required to compute the segmentations
NUC_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'


#instanciation of an object "segmentator"

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cpu",    # to use GPU, check in top right of Kaggle notebook with 3 points, select 'accelerator' and change cpu by "cuda"
    padding=False,
    multi_channel_model=True,
)

In [ ]:
image_id = data_train.loc[0,"ID"] # just need to change the number of image (here row 1)

red = f"../input/hpa-single-cell-image-classification/train/{image_id}_red.png" # the f'string trick avoid to use a {}.format()
green = f"../input/hpa-single-cell-image-classification/train/{image_id}_green.png"
blue = f"../input/hpa-single-cell-image-classification/train/{image_id}_blue.png"

images = [[red], [green], [blue]]

In [ ]:
data_train.loc[0,"Label"]

#### Some useful methods of the HPA CellSegmentator module

- **pred_nuclei :** The function takes a list of image arrays or a list of string paths to images. If the image arrays are 3 channels, the nuclei should be in the third (blue) channel.Returns a list of the neural network outputs of the nuclei segmentation. The images are on the format (3, H, W). The three channels are as follows [<Unused>, touching-nuclei, Nuclei-segmentation].

- **pred_cells :** The function takes a list of three lists as input. The lists should contain either image arrays or string paths, in the order of microtubules, endoplasmic reticulum, and nuclei.
Returns a list of the neural network outputs of the cell segmentations. The images are on the format (3, H, W). The three channels for the cell segmentation are as follows [<Unused>, touching-cells, Cell-segmentation].

Note that both these functions assume that all input images are of the same shape!!
    

- **label_cell :** Input with the nuclei and cell prediction for an image. Returns the labeled nuclei and cell mask arrays as a tuple. As with label_nuclei, the background is 0s and other numbers indicates which cell is there. The same cell will have the same number in both arrays.

In [ ]:
# prediction masks for nuclei mask, return a list of array
nuc_segmentations = segmentator.pred_nuclei(images[2]) # apply segmentator model to the nuclei channel blue corresponding to 2

# prediction masks for full cell, return a list of array
cell_segmentations = segmentator.pred_cells(images) # apply segmentator model to the full object images corresponding to 3 channels RGB 

In [ ]:
nuc_segmentations[0].shape

In [ ]:
cell_segmentations[0].shape

In [ ]:
# post-processing
nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0]) # label_cell is a method of CellSegmentator

In [ ]:
# print the preceding single image(n=1) with 3 channels RGB + mask
# make a function to include number of image
n=0

fig, ax = plt.subplots(1, 2, figsize=(15, 25),subplot_kw=dict(xticks=[], yticks=[]))

red_img = plt.imread(Path(red))    
green_img = plt.imread(Path(green))    
blue_img = plt.imread(Path(blue))

ax[0].imshow(cell_mask, alpha=0.6)
ax[0].set_title("Multi-cell Mask with labels: {}".format(data_train.loc[n,"Label"]), size=16 , color = "green")

img1 = np.dstack((red_img,green_img , blue_img))
ax[1].imshow(img1)
ax[1].set_title("Image RGB with labels: {}".format(data_train.loc[n,"Label"]), size=16 , color = "green")


plt.show()

# 2. More Exploratory Data Analysis (EDA)

## 2.1 Encoding the labels

Recall that our dataset mainly consists of
1. A file train.csv : 21806 images ID
2. A folder train : 21806x4 png files corresponding to channels RGBY, 1 ID = 1 image = 4 channels RGBY

According to the description of the problem, labels may be regrouped by family : 

- Nucleus [0,1,2,3,4,5]  
- Reticulum [6]  
- Tubule [10,11] 
- Cytoplasm [8,9,12,14,15,16] 
- Secretory [7,13,17]

Commentaire: Pour ces familles de labels unique on crée 5 modèles, ces modèles prennent en entrée tableau bleu vert pour predire le noyau, jaune vert pour predire le reticulum, rouge vert pour predire le tubule et rouge vert bleu jaune pour predire les 2 autres classes

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

def binarize_data(df):
    mlb = MultiLabelBinarizer()
    df['labels_list']=[list(map(int, i.split("|"))) for i in df.Label]
    mlb.fit(df['labels_list'].to_list())
    Labels_binarized = mlb.transform(df['labels_list'])
    return Labels_binarized


def label_choice(n):
     data_train['Label_{}'.format(n)]=data_train['Multilabel'].apply(lambda x:x[n])

In [ ]:
# Encoding target into array with multibinarizer (e.g we binarize each label)

multilabel_list = binarize_data(data_train).tolist()
data_train['Multilabel']=[x for x in multilabel_list]
data_train['Multilabel']=data_train['Multilabel'].apply(lambda x:x[0:len(x)-1])
data_train.head()

## 2.2 Splitting our dataset between data_monolabel and data_multilabel

Here we split our train set into mono-label images and multi-label images in order to simplify our problem.

In [ ]:
# Looking distribution of each label in data_train : make graph

number_img_per_target = []
percentage = []
for i in range(18):
    label_choice(i)
    label='Label_{}'.format(i)
    mask=(data_train[label]==1)
    number_img_per_target.append(len(data_train.loc[mask,label]))
    percentage.append(len(data_train.loc[mask,label])/21806)
    percentage = [round(elem,3) for elem in percentage]
print(number_img_per_target)
print(percentage)

In [ ]:
# Spliting data_train between Monolable and Multilabel

data_train["MonoLabel"]=data_train['Multilabel'].apply(lambda x:1 if sum(x)==1 else 0)

data_monolabel=data_train[data_train["MonoLabel"]==1]
data_multilabel=data_train[data_train["MonoLabel"]==0]

#data_multilabel=data_multilabel.sample(100) 
#data_train_croped=data_monolabel #.sample(10000, random_state=42)
#data_train_croped.shape
print("Done--------")

In [ ]:
data_monolabel.head()

In [ ]:
# 10474 images with monolabel
data_monolabel.shape

In [ ]:
''' NUMBER OF IMAGES FOR EACH TARGET Label in data_monolabel'''
liste_i=[]
liste_val=[]
for i in range(18):
    label='Label_{}'.format(i)
    mask=(data_monolabel[label]==1)
    x=data_monolabel.loc[mask,label].sum()
    liste_i.append(i)
    liste_val.append(x)
dico={'x':liste_i,"y":liste_val}
df=pd.DataFrame(dico)
sns.catplot(x='x',y='y', data=df, kind='bar')

In [ ]:
data_multilabel.shape

In [ ]:
data_multilabel.head()

In [ ]:
''' NUMBER OF IMAGES FOR EACH TARGET Label in data_multilabel'''

liste_i=[]
liste_val=[]
for i in range(18):
    label='Label_{}'.format(i)
    mask=(data_multilabel[label]==1)
    x=data_multilabel.loc[mask,label].sum()
    liste_i.append(i)
    liste_val.append(x)
dico={'x':liste_i,"y":liste_val}
df=pd.DataFrame(dico)
sns.catplot(x='x',y='y', data=df, kind='bar')

At this state of the analysis, we remark that the label distribution in mono-label images and multi-label images is highly unbalanced.
Some labels, like label 0 which corresponds to nucleoplasm of the cell is very frequent. On the contrary, some labels are very rare, or even absent like label 11 in 
mono-label images. So this label will be very difficult to detect, as he is only visible in a mixture of other labels. For the training, it will be harder to identify it.

Now we have our images (dataset), we have transformed all the labels (our target) with binary encoding, we have to give this input to some dense neural network in order 
to make some predictions (first at the image-level, not cell level).

### For the next part of this review, we will only study **mono-label images**



## 2.3 Preprocessing our mono-label images dataset

Main steps

- Equilibrate our dataset **data_monolabel** with a best possible balance of labels
- Split into train/validation set (a test set is given apart)




In [ ]:
''' SELECT SUB DATASET WITH ONLY label n'''
def data_monolabel_sub(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==1)
    data_sub = data_monolabel.loc[mask]
    return(data_sub)

''' SELECT SUB DATASET of EQUAL SIZE without label n'''
def data_monolabel_sub_nolabel(n):
    label='Label_{}'.format(n)
    mask=(data_monolabel[label]==0)
    data_sub_nolabel = data_monolabel.loc[mask]
    data_sub_nolabel = data_monolabel.sample(len(data_monolabel_sub(n)), random_state=42)
    return(data_sub_nolabel)

Our global dataset will be the disjoint union (merging) of:
1. data_monolabel_sub(n) = dataset with label n PRESENT in the image ID
2. data_monolabel_sub_nolabel(n) = dataset with label n ABSENT of the image ID

We took same proportions (50/50 percent) in order to train our model. The model will learn equally to detect:

- images with the protein label (n) 
- or images without the protein label (n)

In [ ]:
''' EXECUTE, choose your label number n before'''

n=7 # here this is label 7
data_train_croped = pd.concat([data_monolabel_sub(7),data_monolabel_sub_nolabel(7)], ignore_index=True)
print("Done--------")

In [ ]:
data_train_croped.shape

In [ ]:
data_train_croped=data_train_croped.sample(1000, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
def dataset_split(n): 
    y=data_train_croped['Label_{}'.format(n)]
    X=data_train_croped["ID"]
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,stratify=y)
    return (X_train, X_test, y_train, y_test)

In [ ]:
''' EXECUTE'''

#X_train_0, X_test_0, y_train_0, y_test_0 = dataset_split(0)
X_train_7, X_test_7, y_train_7, y_test_7 = dataset_split(7)
#X_train_9, X_test_9, y_train_9, y_test_9 = dataset_split(9)
#X_train_12, X_test_12, y_train_12, y_test_12 = dataset_split(12)
#X_train_14, X_test_14, y_train_14, y_test_14 = dataset_split(14)
#X_train_15, X_test_15, y_train_15, y_test_15 = dataset_split(15)
#X_train_16, X_test_16, y_train_16, y_test_16 = dataset_split(16)

print("Done--------")

# 3. Preparation of Dataset with Tensorflow

Now we have established our train and test sets, we need to transform the images into suitable format for input into our model (neural network to determine later)

Main steps:

- Transform our images into tensor with Tensorflow
- Decide which neural network we want to use (build from scratch? how deep? Architecture of layers? Using pre-trained model ?...)



In [ ]:
# Because our computations are going to be tedious, we need to clean some variables not useful anymore
del data_train
#del specified_class_names
#del class_names
del multilabel_list
del binarize_data
del label_choice
del data_monolabel
del data_multilabel
del data_train_croped

gc.collect()

print("Done--------")
# Remark :  we can't delete call functions like data_monolabel_sub(n)

## 3.1 Transforming our dataset into Tensorflow.dataset

In [ ]:
'''
##############################################################
# EXECUTE Tensor slices for TRAIN set / label (n)
##############################################################
'''

ds_train_7 = tf.data.Dataset.from_tensor_slices([(open_rgby(str(x))) for x in tqdm(X_train_7)])

In [ ]:
'''
##############################################################
# EXECUTE Tensor slices for VALIDATION set / label (n)
##############################################################
'''
ds_val_7 = tf.data.Dataset.from_tensor_slices([(open_rgby(str(x))) for x in tqdm(X_test_7)])

In [ ]:
'''
##############################################################
# EXECUTE Tensor slices for TARGET of TRAIN set / label (n)
##############################################################
'''
ds_label_7 = tf.data.Dataset.from_tensor_slices(y_train_7)

In [ ]:
'''
##############################################################
# EXECUTE Tensor slices for TARGET of VALIDATION set / label (n)
##############################################################
'''
ds_label_val_7 = tf.data.Dataset.from_tensor_slices(y_test_7)

In [ ]:
# zip train (categorical) and label_train (target)

ds_full_7 = tf.data.Dataset.zip((ds_train_7,ds_label_7))

path_save='./train_ds_full_7'
tf.data.experimental.save(ds_full_7, path_save, compression= 'gzip', shard_func=None)
print("Done--------")

In [ ]:
# zip validation (categorical) and label_validation (target)'''

ds_full_val_7 = tf.data.Dataset.zip((ds_val_7,ds_label_val_7))

path_save='./train_ds_full_val_7'
tf.data.experimental.save(ds_full_val_7, path_save, compression= 'gzip', shard_func=None)
print("Done--------")

In [ ]:
''' TO RELOAD DATA'''

#ds_full_val_7 = tf.data.experimental.load(path_save,tf.TensorSpec(shape=[128,128,4], dtype=tf.uint8), compression='gzip')
#ds_full_val_7

## 3.2 Data augmentation

For RARE labels, that is very few number compared to the number of images, we need to add some data, in order to have 
enough images for training (at least 10 000).
Indeed, some rare labels are present in only a few hundreds or one thousand images, which is too small for training. 
The technique we use here is Data Augmentation, that is, we copy the number of images until the desired number we want for training :
1000 images, copy 10 times --> 10 000 images
each images copy is slightly modified (by some random resizing, brightness, rotate,...) in order to have **different** images for the entire dataset (10 000)
Then, we will train our model for each label on a train set of 10 000 different images containing exactly one label.

In [ ]:
# create fonction which randomly modify each copy
def data_aug_simple(image):
    # rotations and flipping
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, [128, 128, 4])
    image=tf.image.rot90(image, k=1, name=None)
    #image = tf.image.random_brightness(image, 0.05)
    #image = tf.image.random_contrast(image, 0.5, 1.5)
    return image

def data_aug(image, label):
    # rotations and flipping
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, [128, 128, 4])
    image=tf.image.rot90(image, k=1, name=None)
    #image = tf.image.random_brightness(image, 0.05)
    #image = tf.image.random_contrast(image, 0.5, 1.5)
    return image, label


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10), subplot_kw=dict(xticks=[], yticks=[])) 
   
id="000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0"
img=open_rgby(id)
ax[0].imshow(img)
ax[0].set_title('Image (normal)', size=12)
    
img_aug = data_aug_simple(img)
ax[1].imshow(img_aug)
ax[1].set_title('Image (modified)', size=12)

In [ ]:
#REMIND number of images in train set for your label choice (n)
len(ds_full_7)

In [ ]:
# Attention, le nombre de répététion dépend du nombre d'élément dans le data set
#N=7000
#taille=data_train_croped.shape[0]*0.7
#nb_repeat=int(N/len(ds_full))
#nb_repeat


N= 5000 # number of images in input
nb_copy = int(N / len(ds_full_7))
nb_copy

In [ ]:
# Here adapt the number of copy to the size of your dataset data_train_croped

''' EXECUTE TO CREATE FULL DATASET for single label (n)'''
ds_full_7_augmented = ds_full_7.repeat(nb_copy)
ds_full_7_augmented = ds_full_7_augmented.map(data_aug, num_parallel_calls=4) # compute 4 operations in parallel, advise de set this number = CPU number (4)
print("Done--------")

In [ ]:
# check number of images of our full dataset for label n
len(ds_full_7_augmented)

## 3.3 Last step with Tensorflow : Shuffle and Batch Size

In [ ]:
BATCH_SIZE = 32


ds_full_ready_7 = ds_full_7_augmented.shuffle(len(ds_full_7_augmented)).batch(BATCH_SIZE)
ds_full_val_ready_7=ds_full_val_7.batch(BATCH_SIZE) # no need to shuffle for validation set, will not be trained
print("Done--------")

In [ ]:
# checking right format for input of our Neural Network
ds_full_ready_7

In [ ]:
ds_full_ready_7.take(1)

# 4 Definition and training of the model

Now our dataset is suitably prepared as an input to some neural network, we need to find architecture of our model. 
First, it is well know (by experience) that image classification problems require deep neural networks in order to give acceptable results.
It means many layers of different kind and size, the choice of architecture is very complicated. The only thing we now, is that our **last** layer must be 
a dense layer with 18 neurons, as our classification problem has 18 labels (we put label 18 - no protein- aside for the moment).

Over the last few years, there have been a series of breakthroughs in the field of Computer Vision.Especially with the introduction of deep Convolutional neural networks, we are getting state of the art results on problems such as image classification and image recognition. So, over the years, researchers tend to make deeper neural networks(adding more layers) to solve such complex tasks and to also improve the classification/recognition accuracy. But, it has been seen that as we go adding on more layers to the neural network, it becomes difficult to train them and the accuracy starts saturating and then degrades also. Here, the EfficientNet model comes into rescue and helps solve this problem. 

It was first introduced by leading researchers in Google AI in a recent paper (2019) https://paperswithcode.com/paper/efficientnet-rethinking-model-scaling-for
Other reference is https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html


An other excellent choice would have been ResNet, short for Residual Network which is a specific type of neural network that was introduced in 2015 by Microsoft AI researchers Kaiming He, Xiangyu Zhang, Shaoqing Ren and Jian Sun in their paper “Deep Residual Learning for Image Recognition”, in order to solve the problem of the vanishing/exploding gradient. The link to ArXiv is here : https://arxiv.org/abs/1512.03385 

We will not discuss here the foundations and architecture of this model, which is far beyond our scope, but rather take it as a *state-of-the-art* Neural Network model for computer vision.

Due to the enormous number of layers of this NN, we use pre-trained weights for all layers except the last dense layer that we will train to predict our labels.



I'd like to share how to leverage pre-trained 3-channel Keras models to initialize a 4-channel model.

In the discussion forums the competition hosts have stressed the potential importance of all 4 colors, e.g. "All images have all the four channels, and signals from the markers (blue, yellow, red) are present in all cells in the image, independent of the green channel that you are classifying, in order to help you identify where the cells are, as well as where certain structures and regions within the cells are. This can, in turn, help you to segment the cells and to classify each cell to one or more label(s) according to the signal in the green channel." link to the post https://www.kaggle.com/c/hpa-single-cell-image-classification/discussion/215736#1184158



Considering the size of training data, learning a deep 4-channel model with weights initialized at random might be problematic. But **all ImageNet-pre-trained models have 3-channels**. Then, we will initialize a 4-channel EfficientNet with weights reused from a pre-trained 3-channel model.



Main steps :
- load pre-trained weights for the RGB model
- adapt to the RGBY model
- freeze weights in order to train only the last dense layer
- define learning rate schedule, optimizer
- compile the model
- train the model
- display accuracy

In [ ]:
''' EXECUTE'''

# Loading model with weights pre-trained on ImageNet for RGB model
# for Kaggle only : internet must be enabled


DOWNLOAD_PRETRAINED_WEIGHTS = False

weights_init = 'imagenet' if DOWNLOAD_PRETRAINED_WEIGHTS else None

imagenet_model = EfficientNetB0(weights=weights_init, include_top=False, pooling='avg',
                               input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
rgb_model_output = Dense(1, activation='sigmoid')(imagenet_model.output)
model_rgb = Model(inputs=imagenet_model.input, outputs=rgb_model_output)


print("Done--------")

In [ ]:
''' EXECUTE'''

#Model RGBY for label n : just indicate number n after model name: model_rgby_label_n 

four_channel_effnet = EfficientNetB0(weights=None, include_top=False, pooling='avg', 
                                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 4)) # base model
model_rgby_output = Dense( 1, activation='sigmoid')(four_channel_effnet.output)
model_rgby_label_7 = Model(inputs=four_channel_effnet.input, outputs=model_rgby_output)
print("Done--------")
#model_rgby_label_7.summary()

In [ ]:
''' EXECUTE'''

# Loading weights from imagenet model
link_imagenet = '../input/efficientnet-keras-dataset/weights/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
imagenet_model = EfficientNetB0(weights= link_imagenet, include_top=False, pooling='avg', 
                                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
#imagenet_model.summary()
print("Done--------")

In [ ]:
''' EXECUTE'''
# Model RGB from which we load imagenet weigths (3 channels only)
rgb_model_output = Dense(1, activation='sigmoid')(imagenet_model.output)
model_rgb = Model(inputs=imagenet_model.input, outputs=rgb_model_output)
print("Done--------")

In [ ]:
''' EXECUTE'''
# Extending weights from 3 channels to 4 channels
# don't forget to precise the label in the model name: model_rgby_label_n

for layer in tqdm(model_rgby_label_7.layers, desc='Copying the pre-trained net weights..'):
    if 'input' in layer.name or 'dense' in layer.name:
        continue
    elif layer.name == 'stem_conv':
#         with graph_green.as_default():
        kernels = model_rgb.get_layer('stem_conv').get_weights()[0]
        kernels_extra_channel = np.concatenate((kernels, kernels[:,:,:1,:]), axis=-2)
        layer.set_weights([kernels_extra_channel])
    else:
#         with graph_green.as_default():
        weights_green = model_rgb.get_layer(layer.name).get_weights()
        layer.set_weights(weights_green)

In [ ]:
'''EXECUTE'''
#In order to freeze weights from pre-trained model, and train only last Dense layer

four_channel_effnet.trainable = False
print("Done--------")

In [ ]:
'''EXECUTE'''
# Let's create a learning rate schedule to decrease the learning rate as we train the model. 
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)
print("Done--------")

In [ ]:
'''EXECUTE'''

#COMPILE Model label n'''

model_rgby_label_7.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy()]
              )

print("Done--------")

In [ ]:
'''EXECUTE'''
# Clean irrelevant variables e.g all variables != model_rgby_label_n , ds_full_ready_n, ds_full_val_ready_n
# faire une fonction(n)
del ds_train_7
del ds_val_7
del ds_label_7
del ds_label_val_7
del ds_full_7
del ds_full_val_7
del model_rgb
del imagenet_model

gc.collect()

print("Done--------")

In [ ]:
'''
##############################################################
# EXECUTE TRAINING
##############################################################
'''
# Remember to precise your label (n) in model name

history=model_rgby_label_7.fit(ds_full_ready_7, epochs=15, validation_data=ds_full_val_ready_7) # only understand 1 input tensor e.g. list with 1 array 
print("\n ------ \n","training history:",history.history)

In [ ]:
'''EXECUTE'''
#model_rgby_label_2.save("./SAVE_MODEL/model_label_2_monolabel_7000.h5")
#model_rgby_label_6.save("./SAVE_MODEL/model_label_6_monolabel_7000.h5")
#model_rgby_label_7.save("./SAVE_MODEL/model_label_7_monolabel_7000.h5")
#model_rgby_label_10.save("./SAVE_MODEL/model_label_10_monolabel_7000.h5")
#model_rgby_label_13.save("./SAVE_MODEL/model_label_13_monolabel_7000.h5")
#model_rgby_label_17.save("./SAVE_MODEL/model_label_17_monolabel_7000.h5")

model_name = "./SAVE_MODEL/model_label_{}_{}.h5".format(n,N)
model_rgby_label_7.save(model_name)
print("Done--------")

### Conclusion : 
After some tedious computations, we recover the 18 trained models corresponding to label 0 to 17. Here the code save directly the models into a suitable Kaggle folder. To continue this notebook, you'll find all the saved files in the folder "Training" on my Github


In [ ]:
'''EXECUTE only if needed'''
#model_rgby_label_7  = load_model(f'./SAVE_MODEL/model_label7.h5')
#print("Done--------")

In [ ]:
'''EXECUTE'''

from plotly import graph_objects as go
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["binary_accuracy"],
                          name="Training accuracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[4]
                          )),
                      go.Scatter(
                          y=history.history["val_binary_accuracy"],
                          name="Validation accruracy",
                          mode="lines",
                          marker=dict(
                              color=color_chart[5]
                          ))
])
fig.update_layout(
    title='Accuracy / Single label model (label 7) / X images ',
    xaxis_title='epochs',
    yaxis_title='Accuracy'    
)
fig.show()

In [ ]:
'''EXECUTE'''

from plotly import graph_objects as go
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Loss function / Single label model (label 7) / 1000 images',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

# 5. Predictions

In [ ]:
data_multilabel=data_train[data_train["MonoLabel"]==0]
data_multilabel=data_multilabel.sample(100)

data_multilabel['prediction']=data_multilabel['ID'].apply(lambda x: model_rgby.predict(np.expand_dims((open_rgby2(str(x))),0)))

#predictions_test = model_rgby.predict(np.expand_dims((open_rgby2(str(x))) for x in tqdm(data_multilabel['ID']), 0))

In [ ]:
data_multilabel['prediction_lb']=data_multilabel['prediction'].apply(lambda x: 1 if x>0.5 else 0)
                                                                                                    
data_multilabel['r']=data_multilabel['prediction_lb']-data_multilabel['Label_0']
data_multilabel

In [ ]:
data_multilabel['r'].value_counts()

In [ ]:
'''EXECUTE'''

# prediction test for a sample of 100 images
# don't forget to change your label number
n=100
total=0
tester_prediction=X_test_0.sample(n)
tester_prediction=tester_prediction.reset_index()
for i in range (n):
    ID = tester_prediction.loc[i,'ID']
    label=tester_prediction.loc[i,'Label_0']
    img=open_rgby(ID)
    #img=img.astype(np.float32)/255.
    predictions_test = model_rgby_label_0.predict(np.expand_dims(img, 0))
    if label==1 and predictions_test>0.5:
        total=total+1
    #if label==0 and predictions_test<0.5:
    #   total=total+1
    print('prediction=', predictions_test,"attendu:", label)
print('taux succes:',(total/n)*100)

In [ ]:
data_submission_test = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")

In [ ]:
model=[]
N=7000
labels=np.arange(18)
labels
#for df in range(len(sub_dataset)):
    #print('predicting for {}'.format(df))
    
for label in tqdm(labels):    
    model.append(load_model("../input/single-labelmodelsaved/model_label_{}_monolabel_{}.h5".format(label,N)))
    submission_dataset['prediction_{}'.format(label)]=submission_dataset['ID'].apply(lambda x: model[label].predict(np.expand_dims((open_rgby(str(x))),0)))
    submission_dataset['prediction_result_{}'.format(label)]=submission_dataset['prediction_{}'.format(label)].apply(lambda x: 1 if x>0.5 else 0) 
    
#sub_dataset[df].to_csv('./sub_dataset_{}_{}'.format(label,df))